In [58]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

In [5]:
# df = pd.read_csv('Resources/loans_full_schema.csv')
from google.colab import files
uploaded = files.upload()

# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
df = pd.read_csv("loans_full_schema.csv")

# Review the DataFrame
df.head()

Saving loans_full_schema.csv to loans_full_schema.csv


,emp_title,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,annual_income_joint,verification_income_joint,debt_to_income_joint,...,sub_grade,issue_month,loan_status,initial_listing_status,disbursement_method,balance,paid_total,paid_principal,paid_interest,paid_late_fees
0,global config engineer,3.0,NJ,MORTGAGE,90000.0,Verified,18.01,NaN,NaN,NaN,...,C3,Mar-2018,Current,whole,Cash,27015.86,1999.33,984.14,1015.19,0.0
1,warehouse office clerk,10.0,HI,RENT,40000.0,Not Verified,5.04,NaN,NaN,NaN,...,C1,Feb-2018,Current,whole,Cash,4651.37,499.12,348.63,150.49,0.0
2,assembly,3.0,WI,RENT,40000.0,Source Verified,21.15,NaN,NaN,NaN,...,D1,Feb-2018,Current,fractional,Cash,1824.63,281.80,175.37,106.43,0.0
3,customer service,1.0,PA,RENT,30000.0,Not Verified,10.16,NaN,NaN,NaN,...,A3,Jan-2018,Current,whole,Cash,18853.26,3312.89,2746.74,566.15,0.0
4,security supervisor,10.0,CA,RENT,35000.0,Verified,57.96,57000.0,Verified,37.66,...,C3,Mar-2018,Current,whole,Cash,21430.15,2324.65,1569.85,754.80,0.0


In [11]:
column_info = []

for column in df.columns:
      name = df[column].name
      summary_df[name] = df[column].to_frame()
      column_data_type = df[column].dtype
      unique_values = df[column].nunique()
      sample_values = df[column].sample(5).values  # Get 5 sample values
      non_null = df[column].notnull().sum()
      null_values = df[column].isnull().sum()
      null_percent = df[column].isnull().sum()/len(df)  # percent nulls
      column_info.append({
          'Column Name': column,
          'Data Type': column_data_type,
          'Unique Values': unique_values,
          'Non Null': non_null,
          'Null Values': null_values,
          'Null Percent':  null_percent,
          'Sample Values': sample_values
      })

column_summary = pd.DataFrame(column_info)
num_rows, num_columns = df.shape

# Display the summary
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
print("\nColumn Information:")
print(column_summary)

Number of rows: 10000
Number of columns: 55

Column Information:
                         Column Name Data Type  Unique Values  Non Null  \
0                          emp_title    object           4741      9167   
1                         emp_length   float64             11      9183   
2                              state    object             50     10000   
3                      homeownership    object              3     10000   
4                      annual_income   float64           1463     10000   
5                    verified_income    object              3     10000   
6                     debt_to_income   float64           3673      9976   
7                annual_income_joint   float64            596      1495   
8          verification_income_joint    object              3      1455   
9               debt_to_income_joint   float64           1189      1495   
10                         delinq_2y     int64             12     10000   
11          months_since_last_delin

In [12]:
# Fill NA for rows where application type is not "Joint"
df[['verification_income_joint', 'annual_income_joint', 'debt_to_income_joint']] = df[['verification_income_joint', 'annual_income_joint', 'debt_to_income_joint']].fillna('NA')

# Fill NaNs in Days Late Columns with "Not Late"
df['months_since_90d_late'] = df['months_since_90d_late'].fillna('not_late').astype(str)
df['months_since_last_delinq'] = df['months_since_last_delinq'].fillna('not_late_delinq').astype(str)
df['months_since_last_credit_inquiry'] = df['months_since_last_credit_inquiry'].fillna('not_credit_inquiry').astype(str)

# Fill NaNs in Title and Length as No Title and No Length: All Missing Lengths have Not Titles
df['emp_title'] = df['emp_title'].fillna('No Title').astype(str)
df['emp_length'] =df['emp_length'].fillna('No Length').astype(str)

# Fill NaNs in Debt-To-Income with 'Joint"
df['debt_to_income'] = df['debt_to_income'].fillna('joint').astype(str)

# Drop column because no valid data
df.drop('num_accounts_120d_past_due', inplace=True, axis=1)

In [13]:
df.isnull().sum()

emp_title                           0
emp_length                          0
state                               0
homeownership                       0
annual_income                       0
verified_income                     0
debt_to_income                      0
annual_income_joint                 0
verification_income_joint           0
debt_to_income_joint                0
delinq_2y                           0
months_since_last_delinq            0
earliest_credit_line                0
inquiries_last_12m                  0
total_credit_lines                  0
open_credit_lines                   0
total_credit_limit                  0
total_credit_utilized               0
num_collections_last_12m            0
num_historical_failed_to_pay        0
months_since_90d_late               0
current_accounts_delinq             0
total_collection_amount_ever        0
current_installment_accounts        0
accounts_opened_24m                 0
months_since_last_credit_inquiry    0
num_satisfac

In [15]:
df.shape

(10000, 54)

In [16]:
def categorize_risk(status):
    if status in ['Fully Paid', 'Current']:
        return 0
    elif status in ['In grace period', 'Late(31-120days)', 'Late(16-30days)']:
        return 1
    else:
        return 1

# Create new column 'Risk_Category' based on Loan_Status
df['Risk_Category'] = df['loan_status'].apply(categorize_risk)

In [17]:
df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 55 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   emp_title                         10000 non-null  object 
 1   emp_length                        10000 non-null  object 
 2   state                             10000 non-null  object 
 3   homeownership                     10000 non-null  object 
 4   annual_income                     10000 non-null  float64
 5   verified_income                   10000 non-null  object 
 6   debt_to_income                    10000 non-null  object 
 7   annual_income_joint               10000 non-null  object 
 8   verification_income_joint         10000 non-null  object 
 9   debt_to_income_joint              10000 non-null  object 
 10  delinq_2y                         10000 non-null  int64  
 11  months_since_last_delinq          10000 non-null  object 
 12  earli

In [19]:
df.drop('loan_status', inplace=True, axis=1)

In [24]:
# Identify categorical variables in df
categorical_variables = list(df.dtypes[df.dtypes == "object"].index)
numeric_variables = list(df.dtypes[df.dtypes != "object"].index)
display (categorical_variables)
print(" ")
display (numeric_variables)

['emp_title',
 'emp_length',
 'state',
 'homeownership',
 'verified_income',
 'debt_to_income',
 'annual_income_joint',
 'verification_income_joint',
 'debt_to_income_joint',
 'months_since_last_delinq',
 'months_since_90d_late',
 'months_since_last_credit_inquiry',
 'loan_purpose',
 'application_type',
 'grade',
 'sub_grade',
 'issue_month',
 'initial_listing_status',
 'disbursement_method']

['annual_income',
 'delinq_2y',
 'earliest_credit_line',
 'inquiries_last_12m',
 'total_credit_lines',
 'open_credit_lines',
 'total_credit_limit',
 'total_credit_utilized',
 'num_collections_last_12m',
 'num_historical_failed_to_pay',
 'current_accounts_delinq',
 'total_collection_amount_ever',
 'current_installment_accounts',
 'accounts_opened_24m',
 'num_satisfactory_accounts',
 'num_accounts_30d_past_due',
 'num_active_debit_accounts',
 'total_debit_limit',
 'num_total_cc_accounts',
 'num_open_cc_accounts',
 'num_cc_carrying_balance',
 'num_mort_accounts',
 'account_never_delinq_percent',
 'tax_liens',
 'public_record_bankrupt',
 'loan_amount',
 'term',
 'interest_rate',
 'installment',
 'balance',
 'paid_total',
 'paid_principal',
 'paid_interest',
 'paid_late_fees',
 'Risk_Category']

In [ ]:
# df[categorical_variables]
# df.emp_length.value_counts()

In [69]:

df[categorical_variables] = df[categorical_variables].astype(str)

In [31]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)
encoded_data = enc.fit_transform(df[categorical_variables])

In [32]:
encoded_data

array([[0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [33]:
feature_names = enc.get_feature_names_out(categorical_variables)

# Create a new DataFrame with encoded data and appropriate column names
encoded_df = pd.DataFrame(
    encoded_data,
    columns=feature_names
)

In [34]:
encoded_df

,emp_title_ maintenance,emp_title_ admin,emp_title_ combo psc/hub,emp_title_ coordinator,emp_title_ director of engineering,emp_title_ district manager,emp_title_ freight worker,emp_title_ lead hydraulic mechanic,emp_title_ machine operator,emp_title_ quality control technician,...,sub_grade_F5,sub_grade_G1,sub_grade_G4,issue_month_Feb-2018,issue_month_Jan-2018,issue_month_Mar-2018,initial_listing_status_fractional,initial_listing_status_whole,disbursement_method_Cash,disbursement_method_DirectPay
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [36]:
numerical_df = df[numeric_variables]

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit and transform the numerical features
scaled_numerical = scaler.fit_transform(numerical_df)

# Create a DataFrame for scaled numerical features
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_df.columns)

# Concatenate scaled numerical features with encoded features
df_scaled = pd.concat([encoded_df, scaled_numerical_df], axis=1)

In [38]:
df_scaled

,emp_title_ maintenance,emp_title_ admin,emp_title_ combo psc/hub,emp_title_ coordinator,emp_title_ director of engineering,emp_title_ district manager,emp_title_ freight worker,emp_title_ lead hydraulic mechanic,emp_title_ machine operator,emp_title_ quality control technician,...,loan_amount,term,interest_rate,installment,balance,paid_total,paid_principal,paid_interest,paid_late_fees,Risk_Category
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.129752,1.516684,0.328439,0.598041,1.260223,-0.125038,-0.234361,0.803250,-0.065908,-0.13462
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.102945,-0.659333,0.036489,-1.046902,-0.984292,-0.504067,-0.397975,-0.868306,-0.065908,-0.13462
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.394166,-0.659333,0.932336,-1.372981,-1.267985,-0.558973,-0.442581,-0.953479,-0.065908,-0.13462
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.508480,-0.659333,-1.141310,0.637589,0.441019,0.206834,0.219425,-0.064791,-0.065908,-0.13462
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.644383,-0.659333,0.328439,1.053683,0.699638,-0.042846,-0.083569,0.299889,-0.065908,-0.13462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.741457,-0.659333,-1.015331,0.911333,0.715313,0.120152,0.133672,-0.084142,-0.065908,-0.13462
9996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.617576,-0.659333,1.320269,-0.371342,-0.533063,-0.262233,-0.268237,0.007893,-0.065908,-0.13462
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.323900,-0.659333,2.290104,2.370445,1.320619,0.537278,0.125611,3.167942,-0.065908,-0.13462
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.741457,-0.659333,-1.421262,0.836241,0.708466,0.095774,0.151236,-0.402544,-0.065908,-0.13462


In [39]:
df_scaled['Risk_Category'] = df['Risk_Category']

In [48]:
# compute class weight

from sklearn.utils import compute_class_weight

X = df_scaled.drop(columns ='Risk_Category')
y = df_scaled['Risk_Category']

#split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# compute class weights
class_weights = compute_class_weight('balanced', classes= np.unique(y_train), y=y_train)

# Create a dictionary to store the class weights
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# Print the computed class weights
print("Class Weights:", class_weight_dict)


Class Weights: {0: 0.509090909090909, 1: 28.0}


## Use scaled dataframe to train Machine Learning models. RandomForestClassifier

### K-Fold

In [60]:
# Create the Neural Network model
nn_model = Sequential([
    Dense(128, input_dim=X.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')])

# Check the structure of the model
print(nn_model.summary())

# Compile the Sequential model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               1356928   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_1 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                      

In [67]:
# Setup stratified k-fold
stra_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=16)

# Lists to store results of each fold
randf_accuracies = []
randf_class_reports = []
nn_accuracies = []
nn_class_reports = []

# Initialize fold number
fold_no = 1

# for each fold iteration 20% saved for testing and 80% for training
for train_index, test_index in stra_k_fold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # RandomForest Classifier with class weights
    randf_classifier = RandomForestClassifier(random_state=16, class_weight=class_weight_dict)

    randf_classifier.fit(X_train, y_train)
    randf_predictions = randf_classifier.predict(X_test)

    # Evaluating RandomForest Classifier
    randf_accuracy = accuracy_score(y_test, randf_predictions)
    randf_accuracies.append(randf_accuracy)
    randf_class_reports.append(classification_report(y_test, randf_predictions, zero_division=1))

    # Neural Network Model
    nn_model.fit(X_train, y_train)
    nn_predictions = np.argmax(nn_model.predict(X_test), axis=1) # argmax gives labels directly
    nn_accuracy = accuracy_score(y_test, nn_predictions)
    nn_accuracies.append(nn_accuracy)
    nn_class_reports.append(classification_report(y_test, nn_predictions, zero_division=1))

    # Print results for the current fold
    print(f"Results for Fold {fold_no}:")
    print(f"RandomForest Accuracy: {randf_accuracy}")
    print(f"RandomForest Classification Report:\n{classification_report(y_test, randf_predictions, zero_division=1)}")
    print(f"NN Accuracy: {nn_accuracy}")
    print(f"NN Classification Report:\n{classification_report(y_test, nn_predictions, zero_division=1)}")

    # Increment fold number
    fold_no += 1

# Print overall results
print("RandomForest Classifier Overall Performance:")
print("Mean Accuracy:", np.mean(randf_accuracies))
print("Neural Network Overall Performance:")
print("Mean Accuracy:", np.mean(nn_accuracies))

63/63 [==============================] - 0s 6ms/step
Results for Fold 1:
RandomForest Accuracy: 0.9825
RandomForest Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1965
           1       1.00      0.00      0.00        35

    accuracy                           0.98      2000
   macro avg       0.99      0.50      0.50      2000
weighted avg       0.98      0.98      0.97      2000

NN Accuracy: 0.9825
NN Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1965
           1       1.00      0.00      0.00        35

    accuracy                           0.98      2000
   macro avg       0.99      0.50      0.50      2000
weighted avg       0.98      0.98      0.97      2000

63/63 [==============================] - 1s 11ms/step
Results for Fold 2:
RandomForest Accuracy: 0.9825
RandomForest Classification Report:
              precisi

# OLD CODE

In [45]:
# RandomForest Classifier with class weights
rf_classifier = RandomForestClassifier(random_state=42, class_weight= {0:0.0138, 1:(1-0.0138)})
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)

# Evaluating RandomForest Classifier
print("RandomForest Classifier Performance:")
print("Accuracy:", accuracy_score(y_test, rf_predictions))
print("Classification Report:")
print(classification_report(y_test, rf_predictions, zero_division=1))

# Initialize AdaBoostClassifier
adb_classifier = AdaBoostClassifier(random_state=42)

# Train AdaBoostClassifier
adb_classifier.fit(X_train, y_train)

# Make predictions on the test set
adb_predictions = adb_classifier.predict(X_test)

# Evaluate AdaBoostClassifier
adb_accuracy = accuracy_score(y_test, adb_predictions)
print("\nAdaBoost Accuracy:", adb_accuracy)
print("AdaBoost Classification Report:")
print(classification_report(y_test, adb_predictions, zero_division=1))


RandomForest Classifier Performance:
Accuracy: 0.9823333333333333
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2947
           1       1.00      0.00      0.00        53

    accuracy                           0.98      3000
   macro avg       0.99      0.50      0.50      3000
weighted avg       0.98      0.98      0.97      3000


AdaBoost Accuracy: 0.9863333333333333
AdaBoost Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2947
           1       0.88      0.26      0.41        53

    accuracy                           0.99      3000
   macro avg       0.93      0.63      0.70      3000
weighted avg       0.98      0.99      0.98      3000



In [ ]:
# SVM

from sklearn.svm import SVC
svm_clf = SVC(class_weight={0: class_weights[0], 1: class_weights[1]}, random_state=42)
# Train the SVM classifier
svm_clf.fit(X_train, y_train)
# Predictions
y_pred_svm = svm_clf.predict(X_test)


In [ ]:
svm_testing_report = classification_report(y_test, y_pred_svm)
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print(svm_testing_report)


Accuracy: 0.9966666666666667
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2947
           1       0.96      0.85      0.90        53

    accuracy                           1.00      3000
   macro avg       0.98      0.92      0.95      3000
weighted avg       1.00      1.00      1.00      3000



## Next Steps:

1. tuning
2. model validation
3. Neuro Network


Presentation
Slide 1: Title Slide

Title: Machine Learning Classification Project
Subtitle: Predicting Risk Categories
Your Name
Date
Slide 2: Introduction

Brief overview of the project
Objective: Predicting risk categories using machine learning algorithms
Slide 3: Dataset Overview

Description of the dataset used
Number of features and samples
Target variable: Risk Category
Slide 4: Data Preprocessing

Steps involved in data preprocessing
Handling missing values
Encoding categorical variables
Scaling numerical features
Slide 5: Exploratory Data Analysis (EDA)

Visualizations of key features
Distribution of target variable
Correlation analysis
Slide 6: Model Selection

Description of machine learning algorithms used
Random Forest Classifier
AdaBoost Classifier
SVM Classifier
Slide 7: Model Training

Splitting the data into training and testing sets
Training each model on the training set
Brief description of hyperparameters used
Slide 8: Model Evaluation

Evaluation metrics used: Accuracy, Precision, Recall, F1-Score
Performance comparison of each model
Confusion matrices for each model
Slide 9: Results and Conclusion

Summary of the findings
Best performing model
Insights gained from the analysis
Future directions for improvement
Slide 10: Thank You

Acknowledgment
